In [144]:
import pylab
import calendar
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime
from sklearn.preprocessing  import StandardScaler, OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [109]:
raw_data = pd.read_csv("input/train.csv")
test_data = pd.read_csv("input/test.csv")

check data

In [110]:
raw_data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


break datetime to date, hour, weekday, month

In [111]:
def break_datetime(data):
    data['date'] = data.datetime.apply(lambda x: x.split()[0])
    data['hour'] = data.datetime.apply(lambda x: x.split()[1].split(':')[0]).astype('int')
    data['weekday'] = data.date.apply(lambda dateString: datetime.strptime(dateString, '%Y-%m-%d').weekday())
    data['month'] = data.date.apply(lambda dateString: datetime.strptime(dateString, '%Y-%m-%d').month)
    data = data.drop(['datetime', 'date'], axis=1)
    return data

In [112]:
data_no_datetime = break_datetime(raw_data)

In [113]:
data_no_datetime

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,weekday,month
0,1,0,0,1,9.84,14.395,81,0.0000,3,13,16,0,5,1
1,1,0,0,1,9.02,13.635,80,0.0000,8,32,40,1,5,1
2,1,0,0,1,9.02,13.635,80,0.0000,5,27,32,2,5,1
3,1,0,0,1,9.84,14.395,75,0.0000,3,10,13,3,5,1
4,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,7,329,336,19,2,12
10882,4,0,1,1,14.76,17.425,57,15.0013,10,231,241,20,2,12
10883,4,0,1,1,13.94,15.910,61,15.0013,4,164,168,21,2,12
10884,4,0,1,1,13.94,17.425,61,6.0032,12,117,129,22,2,12


change weather and season to one-hot columns

In [114]:
def one_hot_transform(data):
    season_one_hot = OneHotEncoder()
    scol = season_one_hot.fit_transform(data['season'].values.reshape(-1, 1)).toarray()
    scol = pd.DataFrame(scol)
    data = pd.concat([data, scol], axis=1)
    data.drop(['season'], axis=1)

    weather_one_hot = OneHotEncoder()
    wcol = weather_one_hot.fit_transform(data['weather'].values.reshape(-1, 1)).toarray()
    wcol = pd.DataFrame(wcol)
    data = pd.concat([data, wcol], axis=1)
    data.drop(['weather'], axis=1)
    
    return data

In [115]:
data_one_hot = one_hot_transform(data_no_datetime)

In [116]:
data_one_hot

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,...,weekday,month,0,1,2,3,0,1,2,3
0,1,0,0,1,9.84,14.395,81,0.0000,3,13,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,0,0,1,9.02,13.635,80,0.0000,8,32,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,0,0,1,9.02,13.635,80,0.0000,5,27,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,0,0,1,9.84,14.395,75,0.0000,3,10,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,0,0,1,9.84,14.395,75,0.0000,0,1,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,7,329,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10882,4,0,1,1,14.76,17.425,57,15.0013,10,231,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10883,4,0,1,1,13.94,15.910,61,15.0013,4,164,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10884,4,0,1,1,13.94,17.425,61,6.0032,12,117,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


shuffle and split

In [117]:
def split_data(data):
    data = data.sample(frac=1) #shuffle
    test_index = int(data.shape[0] * 0.8)
    y_training = data['count'][:test_index]
    y_validation = data['count'][test_index:]
    data = data.drop(['casual', 'registered', 'count', 'atemp', 'season'], axis=1)
    return data[:test_index], y_training, data[test_index:], y_validation

In [118]:
x_training, y_training, x_validation, y_validation = split_data(data_one_hot)

In [119]:
x_training

,holiday,workingday,weather,temp,humidity,windspeed,hour,weekday,month,0,1,2,3,0,1,2,3
9140,0,1,2,27.06,89,0.0000,5,1,9,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4549,0,1,1,19.68,51,11.0014,15,2,11,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4598,0,1,1,18.86,36,26.0027,16,4,11,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4228,0,0,1,17.22,88,7.0015,5,5,10,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
7532,0,0,1,21.32,72,12.9980,5,6,5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2168,0,1,3,21.32,94,16.9979,6,1,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8662,0,1,1,28.70,74,7.0015,7,4,8,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
6750,0,0,2,18.86,82,11.0014,13,6,3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
256,0,1,2,6.56,86,7.0015,1,2,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [120]:
y_training

9140     38
4549    187
4598    290
4228      8
7532     12
       ... 
2168     49
8662    421
6750    544
256       6
9817    347
Name: count, Length: 8708, dtype: int64

In [121]:
def build_mode(layer_units, input_size):
    layers = []
    layers.append(tf.keras.layers.Dense(units=layer_units[0], activation='relu', input_shape=(input_size,)))
    for i in range(1,len(layer_units)):
        layers.append(tf.keras.layers.Dense(units=layer_units[i], activation='relu'))
    layers.append(tf.keras.layers.Dense(units=1, activation='linear'))
    return tf.keras.models.Sequential(layers=layers)

In [122]:
model = build_mode([17], 17)

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='mse')
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 17)                306       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 18        
Total params: 324
Trainable params: 324
Non-trainable params: 0
_________________________________________________________________


In [123]:
history = model.fit(x_training, y_training, epochs=100, batch_size=128, validation_data=(x_validation, y_validation))

Train on 8708 samples, validate on 2178 samples
Epoch 1/100
8708/8708 [==============================] - 0s 25us/sample - loss: 64299.7586 - val_loss: 57806.6644
Epoch 2/100
8708/8708 [==============================] - 0s 8us/sample - loss: 55009.0552 - val_loss: 46420.7799
Epoch 3/100
8708/8708 [==============================] - 0s 9us/sample - loss: 43916.2664 - val_loss: 37072.7123
Epoch 4/100
8708/8708 [==============================] - 0s 9us/sample - loss: 36793.1004 - val_loss: 32402.6278
Epoch 5/100
8708/8708 [==============================] - 0s 10us/sample - loss: 33406.6880 - val_loss: 30399.3144
Epoch 6/100
8708/8708 [==============================] - 0s 9us/sample - loss: 31884.1402 - val_loss: 29305.5014
Epoch 7/100
8708/8708 [==============================] - 0s 9us/sample - loss: 30723.6551 - val_loss: 28188.7652
Epoch 8/100
8708/8708 [==============================] - 0s 9us/sample - loss: 29445.6379 - val_loss: 26922.3838
Epoch 9/100
8708/8708 [=======================

8708/8708 [==============================] - 0s 10us/sample - loss: 20714.0883 - val_loss: 19760.8469
Epoch 73/100
8708/8708 [==============================] - 0s 10us/sample - loss: 20672.5543 - val_loss: 19727.4049
Epoch 74/100
8708/8708 [==============================] - 0s 10us/sample - loss: 20672.4274 - val_loss: 19695.3791
Epoch 75/100
8708/8708 [==============================] - 0s 11us/sample - loss: 20601.0232 - val_loss: 19642.5828
Epoch 76/100
8708/8708 [==============================] - 0s 10us/sample - loss: 20561.2305 - val_loss: 19642.0364
Epoch 77/100
8708/8708 [==============================] - 0s 10us/sample - loss: 20521.8780 - val_loss: 19575.6839
Epoch 78/100
8708/8708 [==============================] - 0s 9us/sample - loss: 20508.9238 - val_loss: 19532.2251
Epoch 79/100
8708/8708 [==============================] - 0s 9us/sample - loss: 20441.4000 - val_loss: 19509.5856
Epoch 80/100
8708/8708 [==============================] - 0s 8us/sample - loss: 20412.1597 - va

In [ ]:
loss ~ 19574

increase lr

In [126]:
model_lr = build_mode([17], 17)

model_lr.compile(optimizer=tf.keras.optimizers.Adam(lr=0.008), loss='mse')
model_lr.summary()
history = model_lr.fit(x_training, y_training, epochs=100, batch_size=128, validation_data=(x_validation, y_validation))

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 17)                306       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 18        
Total params: 324
Trainable params: 324
Non-trainable params: 0
_________________________________________________________________
Train on 8708 samples, validate on 2178 samples
Epoch 1/100
8708/8708 [==============================] - 0s 25us/sample - loss: 45016.8528 - val_loss: 30423.0082
Epoch 2/100
8708/8708 [==============================] - 0s 8us/sample - loss: 29163.0377 - val_loss: 24079.5859
Epoch 3/100
8708/8708 [==============================] - 0s 9us/sample - loss: 23550.9003 - val_loss: 21094.9832
Epoch 4/100
8708/8708 [==============================] - 0s 9us/sample - loss: 22018.8585 - val_loss: 20859.0315
Epoch 5/100
8708/

8708/8708 [==============================] - 0s 9us/sample - loss: 15532.9497 - val_loss: 15199.5815
Epoch 67/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15624.7907 - val_loss: 15019.0192
Epoch 68/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15655.4889 - val_loss: 14856.6068
Epoch 69/100
8708/8708 [==============================] - 0s 7us/sample - loss: 15612.4520 - val_loss: 14903.2705
Epoch 70/100
8708/8708 [==============================] - 0s 7us/sample - loss: 15541.7694 - val_loss: 14829.9968
Epoch 71/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15637.9990 - val_loss: 15176.0267
Epoch 72/100
8708/8708 [==============================] - 0s 9us/sample - loss: 15707.0150 - val_loss: 14818.3743
Epoch 73/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15490.4812 - val_loss: 15116.8101
Epoch 74/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15554.0538 - val_loss

loss ~ 15161

try scale data

In [127]:
def scale_data(data):
    ss = StandardScaler()
    data = data.values.reshape(-1, 1)
    scaler = ss.fit(data)
    data = ss.transform(data)
    return data, scaler

In [131]:
data_one_hot

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,...,weekday,month,0,1,2,3,0,1,2,3
0,1,0,0,1,9.84,14.395,81,0.0000,3,13,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,0,0,1,9.02,13.635,80,0.0000,8,32,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,0,0,1,9.02,13.635,80,0.0000,5,27,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,0,0,1,9.84,14.395,75,0.0000,3,10,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,0,0,1,9.84,14.395,75,0.0000,0,1,...,5,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10881,4,0,1,1,15.58,19.695,50,26.0027,7,329,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10882,4,0,1,1,14.76,17.425,57,15.0013,10,231,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10883,4,0,1,1,13.94,15.910,61,15.0013,4,164,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10884,4,0,1,1,13.94,17.425,61,6.0032,12,117,...,2,12,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [133]:
scaled_temp, temp_scaler = scale_data(x_training['temp'])
x_scale_temp = x_training.copy()
x_scale_temp['temp'] = scaled_temp
val_scale_temp = temp_scaler.transform(x_validation['temp'].values.reshape(-1,1))
x_val_temp = x_validation.copy()
x_val_temp['temp'] = val_scale_temp
x_scale_temp

,holiday,workingday,weather,temp,humidity,windspeed,hour,weekday,month,0,1,2,3,0,1,2,3
9140,0,1,2,0.869881,89,0.0000,5,1,9,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4549,0,1,1,-0.076695,51,11.0014,15,2,11,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4598,0,1,1,-0.181870,36,26.0027,16,4,11,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4228,0,0,1,-0.392221,88,7.0015,5,5,10,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
7532,0,0,1,0.133655,72,12.9980,5,6,5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2168,0,1,3,0.133655,94,16.9979,6,1,5,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8662,0,1,1,1.080231,74,7.0015,7,4,8,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
6750,0,0,2,-0.181870,82,11.0014,13,6,3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
256,0,1,2,-1.759497,86,7.0015,1,2,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [135]:
model_lr = build_mode([17], 17)

model_lr.compile(optimizer=tf.keras.optimizers.Adam(lr=0.008), loss='mse')
model_lr.summary()
history = model_lr.fit(x_scale_temp, y_training, epochs=100, batch_size=128, validation_data=(x_val_temp, y_validation))

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 17)                306       
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 18        
Total params: 324
Trainable params: 324
Non-trainable params: 0
_________________________________________________________________
Train on 8708 samples, validate on 2178 samples
Epoch 1/100
8708/8708 [==============================] - 0s 25us/sample - loss: 43463.2584 - val_loss: 31984.3873
Epoch 2/100
8708/8708 [==============================] - 0s 9us/sample - loss: 31035.7529 - val_loss: 25767.6986
Epoch 3/100
8708/8708 [==============================] - 0s 11us/sample - loss: 25934.9418 - val_loss: 22987.0167
Epoch 4/100
8708/8708 [==============================] - 0s 10us/sample - loss: 23976.2480 - val_loss: 21946.2155
Epoch 5/100
870

Epoch 67/100
8708/8708 [==============================] - 0s 13us/sample - loss: 15737.4843 - val_loss: 14909.1300
Epoch 68/100
8708/8708 [==============================] - 0s 13us/sample - loss: 15725.5662 - val_loss: 15041.7953
Epoch 69/100
8708/8708 [==============================] - 0s 9us/sample - loss: 15792.1266 - val_loss: 15670.9788
Epoch 70/100
8708/8708 [==============================] - 0s 8us/sample - loss: 16036.0717 - val_loss: 14965.8317
Epoch 71/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15742.7885 - val_loss: 14828.1688
Epoch 72/100
8708/8708 [==============================] - 0s 8us/sample - loss: 15578.3555 - val_loss: 14785.8374
Epoch 73/100
8708/8708 [==============================] - 0s 9us/sample - loss: 15578.5856 - val_loss: 14731.7541
Epoch 74/100
8708/8708 [==============================] - 0s 10us/sample - loss: 15521.2231 - val_loss: 14710.2920
Epoch 75/100
8708/8708 [==============================] - 0s 9us/sample - loss: 15506

loss ~ 14538   better, not much

complex models

In [136]:
model_cplx = build_mode([128, 64], 17) # 1 hidden layer

model_cplx.compile(optimizer=tf.keras.optimizers.Adam(lr=0.008), loss='mse')
model_cplx.summary()
history = model_cplx.fit(x_scale_temp, y_training, epochs=100, batch_size=128, validation_data=(x_val_temp, y_validation))

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 128)               2304      
_________________________________________________________________
dense_23 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 65        
Total params: 10,625
Trainable params: 10,625
Non-trainable params: 0
_________________________________________________________________
Train on 8708 samples, validate on 2178 samples
Epoch 1/100
8708/8708 [==============================] - 0s 27us/sample - loss: 32548.9215 - val_loss: 22028.1403
Epoch 2/100
8708/8708 [==============================] - 0s 12us/sample - loss: 22528.5973 - val_loss: 20209.3316
Epoch 3/100
8708/8708 [==============================] - 0s 11us/sample - loss: 20962.7102 - val_loss: 

Epoch 66/100
8708/8708 [==============================] - 0s 11us/sample - loss: 7190.9081 - val_loss: 6321.2410
Epoch 67/100
8708/8708 [==============================] - 0s 11us/sample - loss: 6908.9159 - val_loss: 7593.2038
Epoch 68/100
8708/8708 [==============================] - 0s 10us/sample - loss: 6838.5689 - val_loss: 14349.3553
Epoch 69/100
8708/8708 [==============================] - 0s 11us/sample - loss: 12132.9441 - val_loss: 8946.9982
Epoch 70/100
8708/8708 [==============================] - 0s 11us/sample - loss: 8100.5705 - val_loss: 7230.0742
Epoch 71/100
8708/8708 [==============================] - 0s 11us/sample - loss: 6906.7819 - val_loss: 9228.3931
Epoch 72/100
8708/8708 [==============================] - 0s 11us/sample - loss: 7798.3061 - val_loss: 8580.4573
Epoch 73/100
8708/8708 [==============================] - 0s 12us/sample - loss: 7901.9064 - val_loss: 7611.2198
Epoch 74/100
8708/8708 [==============================] - 0s 12us/sample - loss: 6597.1333 - v

loss ~ 7189 overfitting after epoch 75. add callback to stop

In [142]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=1, patience=10, verbose=1, mode='min')
model_cplx_cb = build_mode([128, 64], 17) # 1 hidden layer

model_cplx_cb.compile(optimizer=tf.keras.optimizers.Adam(lr=0.008), loss='mse')
model_cplx_cb.summary()
history = model_cplx_cb.fit(x_scale_temp, y_training, epochs=100, batch_size=128, validation_data=(x_val_temp, y_validation), callbacks=[early_stop])

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 128)               2304      
_________________________________________________________________
dense_32 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 65        
Total params: 10,625
Trainable params: 10,625
Non-trainable params: 0
_________________________________________________________________
Train on 8708 samples, validate on 2178 samples
Epoch 1/100
8708/8708 [==============================] - 0s 29us/sample - loss: 30764.9489 - val_loss: 22591.3319
Epoch 2/100
8708/8708 [==============================] - 0s 12us/sample - loss: 22348.9321 - val_loss: 21691.6663
Epoch 3/100
8708/8708 [==============================] - 0s 14us/sample - loss: 20679.7839 - val_loss: 

Epoch 66/100
8708/8708 [==============================] - 0s 12us/sample - loss: 7225.5794 - val_loss: 8644.4055
Epoch 67/100
8708/8708 [==============================] - 0s 11us/sample - loss: 7180.5118 - val_loss: 7331.6716
Epoch 68/100
8708/8708 [==============================] - 0s 11us/sample - loss: 6874.2801 - val_loss: 7533.6175
Epoch 69/100
8708/8708 [==============================] - 0s 11us/sample - loss: 7095.3410 - val_loss: 7977.9810
Epoch 70/100
8708/8708 [==============================] - 0s 11us/sample - loss: 6933.8526 - val_loss: 9094.2130
Epoch 71/100
8708/8708 [==============================] - 0s 10us/sample - loss: 6783.4503 - val_loss: 7488.7709
Epoch 72/100
8708/8708 [==============================] - 0s 10us/sample - loss: 6515.5677 - val_loss: 7347.4599
Epoch 73/100
8708/8708 [==============================] - 0s 11us/sample - loss: 6905.2987 - val_loss: 16452.7931
Epoch 74/100
8708/8708 [==============================] - 0s 11us/sample - loss: 14916.7052 - v

loss ~ 9900

let's save the model

In [145]:
check_point = ModelCheckpoint('/tmp/kaggle-bike/models/model.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_loss', min_delta=1, patience=10, verbose=1, mode='min')
model_cplx_cb_sv = build_mode([128, 64], 17) # 1 hidden layer

model_cplx_cb_sv.compile(optimizer=tf.keras.optimizers.Adam(lr=0.008), loss='mse')
model_cplx_cb_sv.summary()
history = model_cplx_cb_sv.fit(x_scale_temp, y_training, epochs=100, batch_size=128, validation_data=(x_val_temp, y_validation), callbacks=[early_stop, check_point])

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 128)               2304      
_________________________________________________________________
dense_35 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 65        
Total params: 10,625
Trainable params: 10,625
Non-trainable params: 0
_________________________________________________________________
Train on 8708 samples, validate on 2178 samples
Epoch 1/100
6528/8708 [=====================>........] - ETA: 0s - loss: 34712.0546
Epoch 00001: val_loss improved from inf to 22925.00760, saving model to /tmp/kaggle-bike/models/model.hdf5
8708/8708 [==============================] - 0s 31us/sample - loss: 32169.4382 - val_loss: 22925.0076
Epoch 2/100
5760/8708 [===============

Epoch 28/100
5888/8708 [===================>..........] - ETA: 0s - loss: 10256.8066
Epoch 00028: val_loss did not improve from 10238.42718
8708/8708 [==============================] - 0s 11us/sample - loss: 10080.6865 - val_loss: 10790.0907
Epoch 29/100
6528/8708 [=====================>........] - ETA: 0s - loss: 9845.9457
Epoch 00029: val_loss did not improve from 10238.42718
8708/8708 [==============================] - 0s 9us/sample - loss: 10076.2375 - val_loss: 11057.2372
Epoch 30/100
7168/8708 [=======================>......] - ETA: 0s - loss: 10102.3228
Epoch 00030: val_loss did not improve from 10238.42718
8708/8708 [==============================] - 0s 9us/sample - loss: 10098.1030 - val_loss: 11828.0380
Epoch 31/100
6144/8708 [====================>.........] - ETA: 0s - loss: 10391.2796
Epoch 00031: val_loss did not improve from 10238.42718
8708/8708 [==============================] - 0s 9us/sample - loss: 10087.8390 - val_loss: 10756.8650
Epoch 32/100
6016/8708 [============

5760/8708 [==================>...........] - ETA: 0s - loss: 7031.3567
Epoch 00060: val_loss did not improve from 7351.98142
8708/8708 [==============================] - 0s 11us/sample - loss: 7117.2973 - val_loss: 7424.7381
Epoch 61/100
6656/8708 [=====================>........] - ETA: 0s - loss: 7227.2996
Epoch 00061: val_loss did not improve from 7351.98142
8708/8708 [==============================] - 0s 9us/sample - loss: 7222.3915 - val_loss: 7572.3421
Epoch 62/100
5760/8708 [==================>...........] - ETA: 0s - loss: 6937.1318
Epoch 00062: val_loss did not improve from 7351.98142
8708/8708 [==============================] - 0s 10us/sample - loss: 7012.2667 - val_loss: 8657.5528
Epoch 63/100
6144/8708 [====================>.........] - ETA: 0s - loss: 7077.6342
Epoch 00063: val_loss did not improve from 7351.98142
8708/8708 [==============================] - 0s 10us/sample - loss: 7013.9157 - val_loss: 16336.1205
Epoch 00063: early stopping
